[statinfer.com](https://statinfer.com/)

# Packages

In [1]:
import pandas as pd
import sklearn as sk
import numpy as np
import scipy as sp
from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import time
from sklearn.ensemble import GradientBoostingClassifier
import xgboost
from xgboost.sklearn import XGBClassifier
from sklearn import preprocessing

# Data Importing

Dataset Background : https://www.kaggle.com/c/otto-group-product-classification-challenge/notebooks

In [2]:
ecom_prod_train=pd.read_csv("https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Ecom_Products_Menu/train.csv")
ecom_prod_test=pd.read_csv("https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Ecom_Products_Menu/test.csv")

In [ ]:
print(ecom_prod_train.sample(10))
print(ecom_prod_test.sample(10))

In [ ]:
#Frequency counts of the products
ecom_prod_train['Category'].value_counts()

# Prepare Features, Train and Test data

In [ ]:
features=list(ecom_prod_train.columns[1:101])
print("features \n", features)
X_train=ecom_prod_train[features]
y_train=ecom_prod_train['Category']

X_test=ecom_prod_test[features]
y_test=ecom_prod_test['Category']

# Decision Tree model

In [7]:
###buildng Decision tree on the training data ####
from sklearn import tree
tree = tree.DecisionTreeClassifier(max_depth=9)
tree.fit(X_train,y_train)

DecisionTreeClassifier(max_depth=9)

## Decision Tree Results

In [8]:
#Accuracy on train data
print("Decision Tree Results \n")
print("Accuracy on train data" , tree.score(X_train, y_train))
print("Accuracy on test data" , tree.score(X_test, y_test))

Decision Tree Results 

Accuracy on train data 0.6363672638761422
Accuracy on test data 0.6270840421912215


# Boosted Model

VERBOSE=0 : WILL SHOW ONLY END RESULT. NO OF ITERANTIONS =100. TRAINING LOSS = ERROR. REAMNG TIME= TIME LEFT FOR COMPLETE EXECUTION.

In [9]:
from sklearn.ensemble import GradientBoostingClassifier
boost=GradientBoostingClassifier(n_estimators=100,learning_rate=0.1, verbose=1) 
##fitting the gradient boost classifier
start_time = time.time()
boost.fit(X_train,y_train)
print("Time taken by GBM "+ str((time.time() - start_time))+ " Seconds")

      Iter       Train Loss   Remaining Time 
         1           1.6679            2.51m
         2           1.5254            3.81m
         3           1.4205            4.29m
         4           1.3357            4.64m
         5           1.2661            4.63m
         6           1.2069            4.21m
         7           1.1570            3.90m
         8           1.1147            3.66m
         9           1.0746            3.47m
        10           1.0392            3.31m
        20           0.8356            2.59m
        30           0.7398            2.18m
        40           0.6852            1.99m
        50           0.6487            1.62m
        60           0.6220            1.28m
        70           0.6009           58.47s
        80           0.5838           38.04s
        90           0.5694           18.64s
       100           0.5563            0.00s
Time taken by GBM 187.24856185913086 Seconds


## GBM Results

In [10]:
###predicting Gradient boosting model on the train Data
boost_predict_train=boost.predict(X_train)
cm1 = confusion_matrix(y_train,boost_predict_train)
print(cm1)

accuracy_train=f1_score(y_train, boost_predict_train, average='micro') 
print("train accuracy", accuracy_train)

[[ 1497   116    16    11   132    54   162    23   330]
 [   50  6328    91     5    24    74   174     0   117]
 [   21   196  3352     4     6    86   146     1   152]
 [    0     3     3  2144     7     0     4     0    56]
 [  124    18    11     1  2954     7     8    69  3274]
 [   32   225   271     9     5   767   149     2    98]
 [  108   188   126     1    13    50 10813    11   142]
 [   20     7     2    14   264     0    61   934   880]
 [   77    24    12    18  1377     8    32    94 11437]]
train accuracy 0.8025617493316308


In [11]:
###predicting Gradient boosting model on the test Data
boost_predict_test=boost.predict(X_test)
cm1 = confusion_matrix(y_test,boost_predict_test)
print(cm1)

accuracy_test=f1_score(y_test, boost_predict_test, average='micro') 
print("test accuracy", accuracy_test)

[[ 310   31    0    1   25    9   41    5   76]
 [  14 1456   31    1    3   20   49    0   27]
 [   3   41  850    2    0   26   38    0   31]
 [   0    0    0  498    1    0    0    1   22]
 [  21    6    4    0  682    0    2   23  800]
 [   6   68   78    0    4  150   36    0   29]
 [  38   38   37    1    2   10 2512    3   42]
 [   5    1    0    3   68    0   22  169  241]
 [  20    7    7    9  329    0    8   23 2640]]
test accuracy 0.7882783259612114


# XGB Model

PARAMATER : MAX_DEPT = BUILD SHORT TREE. TAKE 50% DEPTH OF ORIGINAL TREE. ETA : STANDARD LEARNIGN RATE. 

In [ ]:
#Creating XGB Friendly data and matrices
train_labels = y_train.values
train_labels = preprocessing.LabelEncoder().fit_transform(train_labels)
test_labels = y_test.values
test_labels = preprocessing.LabelEncoder().fit_transform(test_labels)

matrix_train = xgboost.DMatrix(X_train,label=train_labels)
matrix_test = xgboost.DMatrix(X_test,label=test_labels)

In [ ]:
params = {
    'max_depth': 4,   # SHORTER TREE WILL PROLONG THE ALOGORITHM
    'eta':0.1, #Learning Rate
    'eval_metric':'merror', # Multiclass classification error rate. 
    'tree_method' : "gpu_hist", # use this with colab in gpu mode for faster training
    'num_class': 9
}

start_time = time.time()

model=xgboost.train(params=params,
                    dtrain=matrix_train,
                    num_boost_round=300,    #Number of trees
                    early_stopping_rounds=4, # Stop after 4 rounds, if test error doesn't improve. 
                    evals=[(matrix_test,'test')] 
                   )

print("Time taken by XGB "+ str((time.time() - start_time))+ " Seconds")

## XGB Results

In [ ]:
###prediction using XGB on the train Data
boost_predict_train=model.predict(matrix_train)
cm1 = confusion_matrix(train_labels,boost_predict_train)
print(cm1)

accuracy_train=f1_score(train_labels, boost_predict_train, average='micro') 
print("train accuracy", accuracy_train)

In [ ]:
###prediction using XGB on the test Data
boost_predict_test=model.predict(matrix_test)
cm1 = confusion_matrix(test_labels,boost_predict_test)
print(cm1)

accuracy_test=f1_score(test_labels, boost_predict_test, average='micro') 
print("test accuracy", accuracy_test)